In [0]:
import dlt
from pyspark.sql.functions import from_unixtime, col, round

@dlt.table(
    name="playtime_s",
    comment="Dados playtime convertidos e filtrados para a camada silver"
)
def playtime_silver():
    df = dlt.read("steamdatabricks_workspace.steam.playtime_b")

    df_transformed = (
        df.withColumn('rtime_last_played', from_unixtime(col('rtime_last_played')).cast('timestamp'))
          .withColumn('playtime_forever', round(col('playtime_forever') / 60, 2).cast('double'))
          .filter(col('playtime_forever') > 0.00)
          .withColumn('appid', col('appid').cast('int'))
          .select('appid', 'playtime_forever', 'rtime_last_played')
    )

    return df_transformed